In [1]:
#!pip install mne scipy
#!pip install pandas numpy openpyxl
#!pip install tsfresh
#!pip install PyWavelets

In [2]:
import pandas as pd

# Read time_series_df from CSV
time_series_df = pd.read_csv('time_series_df.csv')

# Read labels from CSV
labels = pd.read_csv('labels.csv')
labels.index = time_series_df['id'].unique()


In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
import gc
from tsfresh import extract_features
from tsfresh.feature_extraction import ComprehensiveFCParameters

# Define the function to process data in chunks using ComprehensiveFCParameters
def process_in_chunks(time_series_df, N):
    # Get unique IDs
    unique_ids = time_series_df['id'].unique()
    
    # Split the unique IDs into chunks of size N
    chunks = [unique_ids[i:i + N] for i in range(0, len(unique_ids), N)]
    
    # Initialize an empty list to store the results
    results = []
    
    # Process each chunk
    for chunk in chunks:
        # Filter the DataFrame to include only the IDs in the current chunk
        chunk_df = time_series_df[time_series_df['id'].isin(chunk)]
        
        # Extract features for the current chunk using ComprehensiveFCParameters
        extracted_features_chunk = extract_features(
            chunk_df,
            column_id='id',
            column_sort='time',
            default_fc_parameters=ComprehensiveFCParameters()
        )
        
        # Append the extracted features to the results list
        results.append(extracted_features_chunk)
        
        # Clear memory
        del chunk_df, extracted_features_chunk
        gc.collect()
    
    # Concatenate all the results into a single DataFrame
    final_result = pd.concat(results)
    
    return final_result

# Set the chunk size N (adjust based on your memory constraints)
N = 10  # Smaller chunk size to manage memory usage

# Extract features using the process_in_chunks function
extracted_features = process_in_chunks(time_series_df, N)

# Drop any columns with NaN or infinite values
extracted_features_clean = extracted_features.replace([np.inf, -np.inf], np.nan).dropna(axis=1)

# Ensure that the labels are aligned with the extracted features
# Assuming 'labels' is a Series with 'id' as the index
labels_aligned = labels.loc[extracted_features_clean.index]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    extracted_features_clean,
    labels_aligned,
    test_size=0.3,
    random_state=42
)

# Select the most important features using ANOVA F-test
selector = SelectKBest(f_classif, k=10)  # Adjust 'k' as needed
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

# Train a Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Define parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
}

# Perform grid search to find the best parameters
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_selected, y_train)

# Print the best parameters found by GridSearchCV
print(f"Best parameters: {grid_search.best_params_}")

# Use the best estimator from GridSearchCV to predict and evaluate the model
best_clf = grid_search.best_estimator_
y_pred = best_clf.predict(X_test_selected)

# Evaluate the model
print(classification_report(y_test, y_pred))

# Identify and display the top selected features with importance
selected_feature_names = extracted_features_clean.columns[selector.get_support()]
important_features = pd.DataFrame({
    'Feature': selected_feature_names,
    'Importance': best_clf.feature_importances_
}).sort_values(by='Importance', ascending=False)

print(important_features)


Feature Extraction: 100%|██████████| 18/18 [00:02<00:00,  7.72it/s]
c:\Users\WERPELGA\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\WERPELGA\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [3] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\WERPELGA\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
c:\Users\WERPELGA\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

Best parameters: {'max_depth': None, 'min_samples_split': 5, 'n_estimators': 100}
              precision    recall  f1-score   support

           0       0.12      0.17      0.14         6
           1       0.38      0.30      0.33        10

    accuracy                           0.25        16
   macro avg       0.25      0.23      0.24        16
weighted avg       0.28      0.25      0.26        16

                     Feature  Importance
8    value__absolute_maximum    0.214460
7             value__maximum    0.168308
9             value__minimum    0.159271
4  value__standard_deviation    0.146875
6    value__root_mean_square    0.134352
0          value__sum_values    0.128472
1              value__median    0.048262
5            value__variance    0.000000
2                value__mean    0.000000
3              value__length    0.000000


c:\Users\WERPELGA\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
